In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población media anual,513170
1,Huelva,Confirmados PDIA,61103
2,Huelva,Confirmados PDIA 14 días,7312
3,Huelva,Tasa PDIA 14 días,"1424,868951809342"
4,Huelva,Confirmados PDIA 7 días,3189
5,Huelva,Tasa PDIA 7 dias,"621,4314944365415"
6,Huelva,Total Confirmados,61320
7,Huelva,Curados,48986
8,Huelva,Fallecidos,429


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  61103.0


/tmp/ipykernel_36171/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  17682.0


/tmp/ipykernel_36171/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]

/tmp/ipykernel_36171/242626003.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_36171/242626003.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_36171/242626003.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 131 personas en los últimos 7 días 

Un positivo PCR cada 60 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población media anual' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Rename columns
tabla.columns = ['Confirmados PDIA', 'Confirmados PDIA 14 días', 'Confirmados PDIA 7 días', 'Población']

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,61103.0,7312.0,3189.0,513170.0,621.431494,1424.868952,435.0
Huelva-Costa,35982.0,4179.0,1916.0,289548.0,661.721027,1443.284015,279.0
Huelva (capital),17682.0,2393.0,1099.0,143837.0,764.059317,1663.688759,170.0
Condado-Campiña,18178.0,1842.0,774.0,156231.0,495.420243,1179.023369,82.0
Sierra de Huelva-Andévalo Central,6494.0,1280.0,494.0,67391.0,733.035569,1899.363416,75.0
Aljaraque,2366.0,328.0,144.0,21474.0,670.578374,1527.428518,39.0
Palos de la Frontera,1508.0,183.0,80.0,11742.0,681.314938,1558.507920,19.0
Isla Cristina,3587.0,257.0,117.0,21393.0,546.907867,1201.327537,17.0
Nerva,383.0,91.0,37.0,5169.0,715.805765,1760.495260,15.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Hinojales,18.0,16.0,15.0,330.0,4545.454545,4848.484848,1.0
Rosal de la Frontera,152.0,64.0,41.0,1697.0,2416.028285,3771.361226,0.0
Campofrío,90.0,26.0,6.0,713.0,841.514727,3646.563815,2.0
Villanueva de las Cruces,41.0,13.0,7.0,387.0,1808.785530,3359.173127,0.0
Jabugo,165.0,69.0,22.0,2260.0,973.451327,3053.097345,7.0
Galaroza,113.0,35.0,7.0,1382.0,506.512301,2532.561505,0.0
San Silvestre de Guzmán,79.0,15.0,4.0,614.0,651.465798,2442.996743,0.0
Valverde del Camino,1445.0,308.0,93.0,12750.0,729.411765,2415.686275,8.0
Aroche,231.0,73.0,32.0,3054.0,1047.806156,2390.307793,4.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Alájar,51.0,7.0,1.0,759.0,131.752306,922.266140,0.0,0.142857
Puerto Moral,27.0,6.0,1.0,278.0,359.712230,2158.273381,0.0,0.166667
Santa Ana la Real,32.0,6.0,1.0,475.0,210.526316,1263.157895,0.0,0.166667
Cumbres Mayores,221.0,22.0,4.0,1749.0,228.702115,1257.861635,2.0,0.181818
Almendro (El),76.0,11.0,2.0,840.0,238.095238,1309.523810,0.0,0.181818
Villarrasa,273.0,20.0,4.0,2211.0,180.913614,904.568069,0.0,0.200000
Galaroza,113.0,35.0,7.0,1382.0,506.512301,2532.561505,0.0,0.200000
Zarza-Perrunal (La),57.0,9.0,2.0,1253.0,159.616919,718.276137,0.0,0.222222
Campofrío,90.0,26.0,6.0,713.0,841.514727,3646.563815,2.0,0.230769
